In [2]:
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import string
import pandas as pd
import glob 
import os
import random
import librosa


In [10]:
def parser(row):
    file_name = os.path.join("six_sec_glass", 'Train ('+ str(row) + ')'+'.wav')
    try:
        Y, sample_rate = librosa.load(file_name, res_type='kaiser_fast',sr=16000,mono=True) 
        pre_emphasis = 2
        X = np.append(Y[0], Y[1:] - pre_emphasis * Y[:-1])
        stft = np.abs(librosa.stft(X,n_fft=512,hop_length=168,win_length=500))
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None
    
    print(row)
    return mfccs,mel,contrast,tonnetz,row


In [3]:
def parser2(row):
    file_name = os.path.join("special_metal", 'metal'+ str(row) + '.wav')
    try:
        Y, sample_rate = librosa.load(file_name, res_type='kaiser_fast',sr=16000,mono=True) 
        pre_emphasis = 2
        X = np.append(Y[0], Y[1:] - pre_emphasis * Y[:-1])
        stft = np.abs(librosa.stft(X,n_fft=512,hop_length=168,win_length=500))
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None
    
    print(row)
    return mfccs,chroma,contrast,tonnetz,row

In [4]:
def feature():
    features=np.empty((0,65))
    label=np.empty((0,1))
    for i in range(1,200):
        mfccs,chroma,contrast,tonnetz,l=parser(i)
        ext_features = np.hstack([mfccs,chroma,contrast,tonnetz])
        features = np.vstack([features,ext_features])
        label=np.vstack((label,1))

    for i in range(1,200):
        mfccs,chroma,contrast,tonnetz,l=parser2(i)
        ext_features = np.hstack([mfccs,chroma,contrast,tonnetz])
        features = np.vstack([features,ext_features])
        label=np.vstack((label,0))
    
    return features,label

In [6]:
f=np.load('f_cry_mfccs,spectral_centroids,spectral_rolloff.npy');
l=np.load('l_cry_mfccs,spectral_centroids,spectral_rolloff.npy');

In [7]:
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor

In [6]:
clf = SVC(kernel='poly',C=0.01,gamma=0.0001, degree=2, probability=True)
clf.fit(f, l.ravel())

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma=0.0001, kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [82]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 100)
rf.fit(f, l.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=100, verbose=0, warm_start=False)

In [8]:
print(l.shape)

(2234, 1)


In [31]:
t, sample_rate = librosa.load('metal1.wav',sr=16000,mono=True) 

In [27]:
pre_emphasis = 2
X = np.append(t[0], t[1:] - pre_emphasis * t[:-1])
stft = np.abs(librosa.stft(X,n_fft=512,hop_length=168,win_length=500))
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)
mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
spectral_centroids = np.mean(librosa.feature.spectral_centroid(X, sr=sample_rate).T,axis=0)
spectral_rolloff =  np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate).T,axis=0)

In [28]:
features_t=np.empty((0,40))
#ext_features_t = np.hstack([mfccs,chroma,mel])
features_t = np.vstack([features_t,mfccs])


In [29]:
print(l.shape)
print(l)

(2198, 1)
[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


In [30]:
clf.predict(features_t)

array([1.])

In [7]:
def parser3(row):
    file_name = os.path.join("six_sec_metal", 'metal ('+ str(row)+')' + '.wav')
    try:
        Y, sample_rate = librosa.load(file_name, res_type='kaiser_fast',sr=16000,mono=True) 
        pre_emphasis = 2
        X = np.append(Y[0], Y[1:] - pre_emphasis * Y[:-1])
        stft = np.abs(librosa.stft(X,n_fft=512,hop_length=168,win_length=500))
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        zcr = np.mean(librosa.feature.zero_crossing_rate(X, frame_length=512, hop_length=168, center=True).T,axis=0) 
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None
    
    print(row)
    return mfccs

In [8]:
def feature_test():
    features=np.empty((0,40))
    y_test=np.empty((0,1))
    for i in range(1,30):
        mfccs=parser3(i)
        features = np.vstack([features,mfccs])
        y_test=np.vstack((y_test,1))
      
    return features,y_test

In [9]:
test_feature,y_test=feature_test()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [10]:
clf.predict(test_feature)
#print(y_test.shape)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [58]:
predictions = rf.predict(test_feature)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.32 degrees.


In [59]:
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 68.24 %.


In [65]:
print(test_feature.shape)
print(y_test.shape)

(699, 180)
(699, 1)


In [11]:
acc = clf.score(test_feature, y_test)
print("acc=%0.5f" % acc)

acc=0.17241


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(f, l.ravel(), test_size=0.4, random_state=0)
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf','poly'], 'gamma': [1e-2,1e-3, 1e-4],'C': [5,10,20],'degree':[2,3]}]
                     
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print('')

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print('')
    print(clf.best_params_)
    print('')
    print("Grid scores on development set:")
    print('')
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
    print('')
    print("Detailed classification report:")
    print('')
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print('')
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print('')

# Tuning hyper-parameters for precision



KeyboardInterrupt: 